In [1]:
import csv
import requests
from bs4 import BeautifulSoup
from random import randint
from time import sleep
import pandas as pd
from tqdm import tqdm

In [2]:
def getHTML(url, page_number = None):
    if page_number != None:
        next_page = url + str(page_number)
    else:
        next_page = url
    response = requests.get(str(next_page), timeout = 5)
    soup = BeautifulSoup(response.content,'html.parser')
    return response, soup

In [6]:
def linkedInScrapper():      
    job = input('Insert the desired job: ')
    location = input('Insert the desired location: ')
    job_limit = input('Choose the maximum number of jobs to search (multiples of 25): ')
    if len(job.split()) == 1:
        if len(location.split()) == 1:
            url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=' + job + '&location=' + location + '&geoId=100364837&trk=public_jobs_jobs-search-bar_search-submit&currentJobId=3507693000&position=1&pageNum=0&start='
        else:
            location_list = location.split()
            full_location = ''
            for i in range(len(location_list)-1):
                full_location += location_list[i] + '+'
            full_location += location_list[len(location_list)-1]
            url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=' + job + '&location=' + full_location + '&geoId=100364837&trk=public_jobs_jobs-search-bar_search-submit&currentJobId=3507693000&position=1&pageNum=0&start='
    else:
        job_list = job.split()
        full_job = ''
        for i in range(len(job_list)-1):
            full_job += job_list[i] + '+'
        full_job += job_list[len(job_list)-1]
        if len(location.split()) == 1:
            url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=' + full_job + '&location=' + location + '&geoId=100364837&trk=public_jobs_jobs-search-bar_search-submit&currentJobId=3507693000&position=1&pageNum=0&start='
        else:
            location_list = location.split()
            full_location = ''
            for i in range(len(location_list)-1):
                full_location += location_list[i] + '+'
            full_location += location_list[len(location_list)-1]
            url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=' + full_job + '&location=' + full_location + '&geoId=100364837&trk=public_jobs_jobs-search-bar_search-submit&currentJobId=3507693000&position=1&pageNum=0&start='
    
    current_limit = 0
    url_list = []
    count = 0
    response, soup = getHTML(url, current_limit)
    
    file = open('linkedin_' + job + '_' + location + '.csv', 'w')
    writer = csv.writer(file)
    writer.writerow(['Title', 'Company', 'Location', 'URL', 'Description'])
    
    with tqdm(total = int(job_limit)) as pbar:
        while '200' in str(response) and current_limit < int(job_limit):
            jobs = soup.find_all('div', class_='base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card')
            for job in jobs:
                job_title = job.find('h3', class_='base-search-card__title').text.strip()
                company = job.find('h4', class_='base-search-card__subtitle').text.strip()
                location = job.find('span', class_='job-search-card__location').text.strip()
                job_url = job.find('a', class_='base-card__full-link')['href']
            
                if job_url not in url_list:
                    url_list.append(job_url)
                    sleep(randint(1,5))
                    try:
                        response_job, soup_job = getHTML(job_url)
                        description = soup_job.find('div', class_='show-more-less-html__markup').text.strip()
                        writer.writerow([job_title, company, location, job_url, description])
                    except:
                        continue
                else:
                    continue
                
                count += 1
                pbar.update(1)
                
                #print('\nJob title:', job_title, '\nCompany name:', company, '\nJob location:', location, '\nJob URL:', job_url, '\nDescription:', description)
        
            current_limit += 25
            sleep(randint(1,5))
            response, soup = getHTML(url, current_limit)
        
        pbar.update(int(job_limit)-count)
        file.close()

In [7]:
linkedInScrapper()

Insert the desired job: Analista de Dados
Insert the desired location: Portugal
Choose the maximum number of jobs to search (multiples of 25): 50


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [03:07<00:00,  3.74s/it]


In [8]:
dataframe = pd.read_csv('linkedin_Analista de Dados_Portugal.csv', encoding = 'latin-1')
dataframe

,Title,Company,Location,URL,Description
0,Data Analyst,"VASP - Distribuidora de Publicações, S.A.","Agualva, Lisbon, Portugal",https://pt.linkedin.com/jobs/view/data-analyst...,Empresa líder no mercado de Distribuição de Pu...
1,Data Analyst/Analista de BI,NEX T Engineering,"Lisbon, Portugal",https://pt.linkedin.com/jobs/view/data-analyst...,Your NEX T step is right here!Procuramos quali...
2,DATA ANALYST (remoto Brasil),iTRecruiter,"Lisboa, Lisbon, Portugal",https://pt.linkedin.com/jobs/view/data-analyst...,DATA ANALYSTConsultoria portuguesa contrata pa...
3,Data Analyst,AdvanceCare,"Lisbon, Portugal",https://pt.linkedin.com/jobs/view/data-analyst...,Somos uma empresa de prestação de serviços na ...
4,Data Analyst,Devoteam,"Porto, Portugal",https://pt.linkedin.com/jobs/view/data-analyst...,"ABOUT DEVOTEAMAt Devoteam, we believe that tec..."
5,Data Analyst,radicant,"Lisbon, Portugal",https://pt.linkedin.com/jobs/view/data-analyst...,Your opportunity with a higher purposeBecome p...
6,Data Analyst- Porto or Lisbon,BNP Paribas,"Lisboa, Lisbon, Portugal",https://pt.linkedin.com/jobs/view/data-analyst...,About The Job The Client Development team supp...
7,Data Analyst - Lisboa - Campo Grande,EGOR,"Lisboa, Lisbon, Portugal",https://pt.linkedin.com/jobs/view/data-analyst...,A EGOR OUTSOURCING admite Analista de Dados co...
8,Data Analyst / Analytics Engineer,Trouva,"Lisboa, Lisbon, Portugal",https://pt.linkedin.com/jobs/view/data-analyst...,DescriptionTrouva is the leading platform to d...
9,Data Analyst,Winning,"Lisbon, Lisbon, Portugal",https://pt.linkedin.com/jobs/view/data-analyst...,A Winning está à procura de uma Data Analyst c...
